In [216]:
%%capture
%run 04_account.ipynb
%run 05_block.ipynb

## Miner

Mining is a crucial component on any blockchain relying on proof of work (abbr. pow). It is used to reach consensus in a network of anonymous decentralized nodes. Anyone with an account `acc` can become a miner. 

The miner takes a new block and tries to solve a specific computational puzzle, a specific hash. This can only be done by brute force. 

`coinbase` creates a tx to the miner as a reward for providing the pow (also called coinbase tx). 

In [217]:
class Miner:
    def __init__(self, acc): 
        self.acc = acc
        self.pub = acc.pub
        
    def mine(self, txs, prev_header, diff, reward, attempts=1000):
        txs.insert(0, self.coinbase(reward))
        mt = MerkleTree(txs)
        bh = Header(mt.root, prev_header.hash, prev_header.number+1, len(txs))
        bh.diff   = diff
        bh.reward = reward
        bh.miner  = self.pub
        bh.mined  = True
        bh_b = bytes(bh)
        nonce = 0
        for i in range(attempts):
            candidate   = bh_b + str(nonce).encode()
            candidate_h = sha(candidate)
            if candidate_h[2:2+diff] == '0'*diff: break
            nonce += 1
            assert nonce != attempts, 'no nonce could be found'
        bh.nonce  = nonce
        return Block(bh, txs)
        
    def coinbase(self, reward): 
        return self.acc.sign(TX(self.pub, self.pub, reward, 0, self.acc.nonce))

### Genesis Block

Is the first block in the blockchain. This is how it all begins. 

In [227]:
def mine_genesis(txs):
    mt  = MerkleTree(txs)
    bh  = Header(mt.root, '0x0', 0, len(txs))
    return Block(bh, txs)

gb  = mine_genesis(r_stxs(2)); print(gb)

root:          🔖 0xb08d037053...f3b
hash:          📚 0x74d0d1938d...7d7
prev_hash:     0x0
number:        0
n_txs:         2
mined:         False
time:          Wed Mar 31 01:04:32 2021
volume:        9.0 ether
fees:          0.16 ether
sz:            2422

txs:
👯 0x097008cf7c...a2f
📳 0x8d2acb877f...f9a


### Mining

Mine a block on top of the genesis block

In [220]:
miner = Miner(acc1)
mb = miner.mine(r_stxs(10), gb.header, 1, 100); print(mb)

root:          💜 0x3641bb8df5...226
hash:          📳 0x8ddace8ecf...99f
prev_hash:     💆 0x2082efb5d5...86b
number:        1
n_txs:         11
mined:         True
time:          Wed Mar 31 00:59:35 2021
diff:          1
reward:        100
miner:         💚 0x34B23f22C5...621
nonce:         0
volume:        156.0 ether
fees:          0.59 ether
sz:            11768

txs:
🔮 0xc8eb0f058e...5e8
🔐 0xaa4e5f1d41...c6e
💄 0x1e34bb5a34...46c
💁 0x1b7d39c5e6...369
🔳 0xcd9b4a22b3...d37
🔿 0xd9bf87a193...140
🕐 0xead848ea16...af8
🔵 0xcf4dbfd3c5...a81
💆 0x208c588124...adc
💫 0x451a574637...0a6
🕄 0xde7051b839...704


### PoW validation

We can validate a mined blocks pow very easily.

In [222]:
def val_pow(mb):
    bh          = mb.header
    mb_b        = bytes(bh)
    candidate   = mb_b + str(bh.nonce).encode()
    candidate_h = sha(candidate)
    if candidate_h[2:2+bh.diff] == '0'*bh.diff: return True
    else                                      : return False
    
assert val_pow(mb)

If we change something in the block the pow should break.

In [224]:
mb.header.prev_hash = rh()
assert not val_pow(mb)